In [29]:
import re
import numpy as np
from collections import defaultdict


# 创建一个嵌套的defaultdict
def nested_defaultdict():
    return defaultdict(nested_defaultdict)


# 适用于eval4附带效用值的数据
def parse_epoch_data2(file_path):
    with open(file_path, "r") as file:
        data = file.read()
    # 正则表达式模式匹配EPOCH数据
    epoch_pattern = re.compile(
        r"Epoch (\d+): Time ([\d.]+) s, model acc: ([\d.]+)\n"
        r"(\w+): ([\d.]+) s, acc ([\d.]+), f1 ([\d.]+)\n"
        r"mo_utility: ([\d., -]+)\n"
        r"participant_utilities: \[([\d., -]+)\]\n"
        r"--------------------------"
    )

    epochs = []

    for match in epoch_pattern.finditer(data):
        try:
            participant_utilities = list(map(float, match.group(9).split(", ")))

            epoch_data = {
                "epoch": int(match.group(1)),
                "time": float(match.group(2)),
                "model_acc": float(match.group(3)),
                "veri_method": match.group(4),
                "veri_time": float(match.group(5)),
                "veri_acc": float(match.group(6)),
                "veri_f1": float(match.group(7)),
                "mo_utility": float(match.group(8)),
                "participant_utilities": participant_utilities,
            }
            epochs.append(epoch_data)
        except (ValueError, IndexError) as e:
            print(f"Error parsing line: {match.group()}. Error: {e}")

    return epochs


# 适用于eval1-3不附带效用值的数据
def parse_epoch_data1(file_path):
    with open(file_path, "r") as file:
        data = file.read()

    # 正则表达式模式匹配EPOCH数据
    epoch_pattern = re.compile(
        r"Epoch (\d+): Time ([\d.]+) s, model acc: ([\d.]+)\n"
        r"(\w+): ([\d.]+) s, acc ([\d.]+), f1 ([\d.]+)\n"
        r"--------------------------"
    )

    epochs = []

    for match in epoch_pattern.finditer(data):
        epoch_data = {
            "epoch": int(match.group(1)),
            "time": float(match.group(2)),
            "model_acc": float(match.group(3)),
            "veri_method": match.group(4),
            "veri_time": float(match.group(5)),
            "veri_acc": float(match.group(6)),
            "veri_f1": float(match.group(7)),
        }
        epochs.append(epoch_data)

    return epochs


def read_file(base_path, tasks, veri_methods, maricious_nums):
    match = re.search(r"eval(\d+)", base_path)
    eval_num = int(match.group(1))
    # 实验结果路径
    ret = nested_defaultdict()
    for task in tasks:
        for veri_method in veri_methods:
            for maricious_num in maricious_nums:
                # 解析实验结果
                file_path = (
                    base_path
                    + f"{task}-{veri_method}-adv{maricious_num[0]}-fr{maricious_num[1]}"
                )
                if eval_num in [1, 2, 3]:
                    epochs_data = parse_epoch_data1(file_path)
                else:
                    epochs_data = parse_epoch_data2(file_path)
                ret[task][veri_method][maricious_num[0]][maricious_num[1]] = (
                    (task, veri_method, maricious_num[0], maricious_num[1]),
                    epochs_data,
                )
    return ret

In [30]:
# TODO: 提取各实验model acc并对比(思考直观的展示方法)
# 控制变量，相同任务+攻击，对比不同验证方法

# TODO：思考各参与者效用值展示方式

tasks = ["CIFAR10", "MNIST"]
veri_methods = [
    "none",
    "test_acc",
    # "shapley",
    "influence",
    "multi_KRUM",
    "update_significance",
    "reproduction",
]
maricious_nums = [(0, 2), (0, 4), (2, 0), (4, 0), (2, 2)]

base_path = "./results/eval6/"
datas = read_file(base_path, tasks, veri_methods, maricious_nums)

for task in tasks:
    for maricious_num in maricious_nums:
        for veri_method in veri_methods:
            data = datas[task][veri_method][maricious_num[0]][maricious_num[1]]
            meta_data = data[0]
            epoch_data = data[1]
            max_acc = max([line["model_acc"] for line in epoch_data])

            # TODO: 输出攻击者和诚实者的平均效用
            attacker_num = maricious_num[0] + maricious_num[1]
            attacker_avg_utility = sum(
                [
                    np.mean(line["participant_utilities"][:attacker_num])
                    for line in epoch_data
                ]
            )
            # 计算攻击者平均效用
            normal_avg_utility = sum(
                [
                    np.mean(line["participant_utilities"][attacker_num:])
                    for line in epoch_data
                ]
            )
            # 计算诚实参与者平均效用
            total_mo_utility = sum([line["mo_utility"] for line in epoch_data])
            print(
                meta_data,
                max_acc,
                f"mo_u {total_mo_utility:.2f}",
                f"atk_u {attacker_avg_utility:.2f}",
                f"normal_u {normal_avg_utility:.2f}",
            )
        print("--------------")

('CIFAR10', 'none', 0, 2) 73.95 mo_u -6221.71 atk_u 1234.80 normal_u 439.60
('CIFAR10', 'test_acc', 0, 2) 73.35 mo_u -1793.96 atk_u 912.51 normal_u -33.30
('CIFAR10', 'influence', 0, 2) 72.32 mo_u -263.37 atk_u 1127.37 normal_u -278.33
('CIFAR10', 'multi_KRUM', 0, 2) 75.21 mo_u 2372.69 atk_u -3062.40 normal_u 439.60
('CIFAR10', 'update_significance', 0, 2) 75.81 mo_u 2372.69 atk_u -3062.40 normal_u 439.60
('CIFAR10', 'reproduction', 0, 2) 74.82 mo_u 2372.69 atk_u -3062.40 normal_u 439.60
--------------
('CIFAR10', 'none', 0, 4) 69.5 mo_u -15720.00 atk_u 1095.60 normal_u 267.33
('CIFAR10', 'test_acc', 0, 4) 67.96 mo_u -9740.84 atk_u 773.31 normal_u -514.33
('CIFAR10', 'influence', 0, 4) 65.65 mo_u -10427.00 atk_u 1095.60 normal_u -614.83
('CIFAR10', 'multi_KRUM', 0, 4) 72.28 mo_u 1468.80 atk_u -3201.60 normal_u 267.33
('CIFAR10', 'update_significance', 0, 4) 72.19 mo_u 1468.80 atk_u -3201.60 normal_u 267.33
('CIFAR10', 'reproduction', 0, 4) 72.41 mo_u 1468.80 atk_u -3201.60 normal_u 267